In [249]:
import torch
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

In [297]:
# helper.py

# Count the number of parameters
def count_param(model):
    return sum([torch.numel(param) for param in model.parameters()])

In [327]:
# data_loader.py
from torch.utils.data import TensorDataset, DataLoader

def load_data(N=1000, batch_size=50, seed=42):
    # Load data
    train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)
    
    # Normalize data
    mean, std = train_input.mean(), train_input.std()
    train_input.sub_(mean).div_(std)
    test_input.sub_(mean).div_(std)
    
    # Generate dataset
    train_data = TensorDataset(train_input, train_target, train_classes)
    test_data = TensorDataset(test_input, test_target, test_classes)
    
    # For reproducibility
    torch.manual_seed(seed)
    
    # Generate data loader
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size)
    
    return train_loader, test_loader

In [328]:
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=42)

In [329]:
def compute_nb_errors(model, data_loader):

    nb_data_errors = 0

    for data_input, data_target, data_classes in data_loader:
        data_target = torch.nn.functional.one_hot(data_target)
        output = model(data_input)
        nb_error = torch.sum(torch.argmax(output, dim=1, keepdim=True) != torch.argmax(data_target, dim=1, keepdim=True))
        nb_data_errors += nb_error
        
    return nb_data_errors

In [358]:
class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=5)    # size [nb, 32, 10, 10]
        self.conv2 = nn.Conv2d(32, 64, kernel_size=2)   # size [nb, 64, 4, 4]
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(10, 2)
        
    def forward(self, x):        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2)) # size [nb, 32, 5, 5]      
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) # size [nb, 64, 2, 2]
        x = x.view(-1, 256) # size [nb, 256]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [331]:
BaseNet = BaseNet()

# Calculate the number of parameters in the model
count_param(BaseNet)

63320

In [367]:
def train(model, train_loader, eta, n_epochs=25, verbose=False):

    binary_crit = nn.CrossEntropyLoss()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=eta)

    tr_losses = []
    tr_accuracies = []

    for e in range(n_epochs):
        # Reset training/validation loss
        tr_loss = 0

        # Training model
        model.train()

        for train_input, train_target, train_classes in iter(train_loader):
            # Forward pass
            output = model(train_input)

            # Binary classification loss
            binary_loss = binary_crit(output, train_target)
        
            # Total loss = Binary loss
            tr_loss += binary_loss

            # Backward pass
            optimizer.zero_grad()
            binary_loss.backward()
            optimizer.step()

        # Collect accuracy data
        tr_accuracies.append(compute_nb_errors(model, train_loader)/1000)

        # Collect loss data
        tr_losses.append(tr_loss)

        if verbose:
            print('Epoch %d/%d, Binary loss: %.3f' %
                  (e+1, n_epochs, tr_loss))

In [372]:
import time

time1 = time.perf_counter()
model = BaseNet()
train(model, train_loader, 0.001, 25, verbose=False)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

Spend 7.052693e+00 s
tensor(0.9850) tensor(0.8090)


In [370]:
accuracies = []
model = BaseNet()
for i in range(10):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=42)
    train(model, train_loader, 0.001, 25, verbose=False)
    te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
    accuracies.append(te_accuracy)
print(accuracies, torch.Tensor(accuracies).mean(), torch.Tensor(accuracies).std())

[tensor(0.8240), tensor(0.8310), tensor(0.8280), tensor(0.8300), tensor(0.8270), tensor(0.8190), tensor(0.8210), tensor(0.8160), tensor(0.8190), tensor(0.8060)] tensor(0.8221) tensor(0.0076)
